In [1]:
! pip install modin[all]

In [2]:
import modin.pandas as pd
import matplotlib.pyplot as plt

import numpy as np
import seaborn as sns
import time
from sklearn.metrics import accuracy_score,recall_score,precision_score,f1_score,confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.preprocessing import LabelEncoder


from sklearnex import patch_sklearn
patch_sklearn()



Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


In [3]:

data = pd.read_csv("Toddler Autism dataset July 2018.csv")
data=data.drop("Case_No",axis=1)


pd.set_option('display.max_columns', None)


d1 = pd.read_csv("Toddler Autism dataset July 2018.csv")
d1["Age_Mons"]=(d1["Age_Mons"]/12).astype(int)


d2 = pd.read_csv("data_csv.csv")
d2=d2.dropna()


d3 = pd.read_csv("autism_screening.csv")

d3=d3.dropna()
d3["age"]=(d3["age"]/12).astype(int)



d1=d1.iloc[:,1:]
d2=pd.concat([d2.iloc[:,1:11],d2.iloc[:,[12,13,22,23,24,25,26,27]]],axis=1)
d3=pd.concat([d3.iloc[:,0:11],d3.iloc[:,[17,11,12,13,14,19,20]]],axis=1)


d1.columns = d2.columns
d3.columns = d2.columns


2024-02-25 17:40:43,366	INFO worker.py:1724 -- Started a local Ray instance.
Data types of partitions are different! Please refer to the troubleshooting section of the Modin documentation to fix this issue.


In [4]:
data=pd.concat([d1,d2,d3],axis=0)



replacements = {
    'f': 'F',
    'm': 'M',
}
data['Sex'] = data['Sex'].replace(replacements)
replacements = {
    'yes': 'Yes',
    'no': 'No',
}
data['Jaundice'] = data['Jaundice'].replace(replacements)
replacements = {
    'yes': 'Yes',
    'no': 'No',
}
data['Family_mem_with_ASD'] = data['Family_mem_with_ASD'].replace(replacements)
replacements = {
    'YES': 'Yes',
    'NO': 'No',
}
data['ASD_traits'] = data['ASD_traits'].replace(replacements)

replacements = {
    'middle eastern': 'Middle Eastern',
    'Middle Eastern ': 'Middle Eastern',
    'mixed': 'Mixed',
    'asian': 'Asian',
    'black': 'Black',
    'south asian': 'South Asian',
    'PaciFica':'Pacifica',
    'Pasifika':'Pacifica'

}
data['Ethnicity'] = data['Ethnicity'].replace(replacements)

replacements = {
    'Health care professional':'Health Care Professional',
    'family member':'Family Member',
    'Family member':'Family Member'
}
data['Who_completed_the_test'] = data['Who_completed_the_test'].replace(replacements)



In [5]:


X = data.drop("ASD_traits",axis=1)
y = data["ASD_traits"]


# One Hot Encoding
X=pd.get_dummies(X,columns=["Ethnicity", "Who_completed_the_test"], drop_first=True)

# Label Encoding
le=LabelEncoder()
y=le.fit_transform(y)
X['Sex']=le.fit_transform(X['Sex'])
X['Jaundice']=le.fit_transform(X['Jaundice'])
X['Family_mem_with_ASD']=le.fit_transform(X['Family_mem_with_ASD'])

print(X)

"""# Splitting of Data"""


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state =1693)

"""# Feature Scaling"""

sc=StandardScaler()
X_train_scaled=sc.fit_transform(X_train)
X_test_scaled=sc.transform(X_test)


     A1  A2  A3  A4  A5  A6  A7  A8  A9  A10_Autism_Spectrum_Quotient  \
0     0   0   0   0   0   0   1   1   0                             1   
1     1   1   0   0   0   1   1   0   0                             0   
2     1   0   0   0   0   0   1   1   0                             1   
3     1   1   1   1   1   1   1   1   1                             1   
4     1   1   0   1   1   1   1   1   1                             1   
..   ..  ..  ..  ..  ..  ..  ..  ..  ..                           ...   
699   0   1   0   1   1   0   1   1   1                             1   
700   1   0   0   0   0   0   0   1   0                             1   
701   1   0   1   1   1   0   1   1   0                             1   
702   1   0   0   1   1   0   1   0   1                             1   
703   1   0   1   1   1   0   1   1   1                             1   

     Age_Years  Qchat_10_Score  Sex  Jaundice  Family_mem_with_ASD  \
0            2             3.0    0         1        

In [6]:

def train_model(model, X_train, y_train, X_test, y_test):
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    matrix=confusion_matrix(y_test,y_pred)
    print("Confusion Matrix is: ")
    print(matrix)
    print("\n\nScore Table is: ")
    score_df = pd.DataFrame([[accuracy, precision, recall, f1]],
                            columns=["accuracy", "precision", "recall", "f1"])

    return score_df

In [7]:
# XGBoost Classifier
from xgboost import XGBClassifier

model = XGBClassifier()
t0 = time.time()
# training the model for prediction
result = train_model(model, X_train_scaled, y_train, X_test_scaled, y_test)
t1 = time.time()
d4p_total = t1 - t0
# Your result calculation
result.index = ["XGBoost Classifier"]
print(result)

# Print daal4py execution time
print(f"daal4py Linear Regression Execution time: {d4p_total} seconds")

#result = result.append(xg)
print(result)


Confusion Matrix is: 
[[427   5]
 [  2 486]]


Score Table is: 
                    accuracy  precision    recall       f1
XGBoost Classifier  0.992391   0.989817  0.995902  0.99285
daal4py Linear Regression Execution time: 0.10769939422607422 seconds
                    accuracy  precision    recall       f1
XGBoost Classifier  0.992391   0.989817  0.995902  0.99285


In [8]:
import joblib
# GradientBoostingClassifier
model = GradientBoostingClassifier()
t0 = time.time()
result1 = train_model(model, X_train_scaled, y_train, X_test_scaled, y_test)
t1 = time.time()
elapsed_time = t1 - t0
result1.index = ["Gradient Boosting Classifier"]
print(result1)

print(f"Execution time for script2: {elapsed_time} seconds")

Confusion Matrix is: 
[[427   5]
 [  1 487]]


Score Table is: 
                              accuracy  precision    recall        f1
Gradient Boosting Classifier  0.993478   0.989837  0.997951  0.993878
Execution time for script2: 0.37957191467285156 seconds
